## 라이브러리

In [39]:
import os
import platform
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

## 경로설정

In [49]:
# --------------------------------------------------------
# 7번째 줄
root_dir = os.path.abspath(os.curdir)
print(root_dir) # d:\yespeechcode\kochat\yr_hack
# 현재 있는 폴더 경로 반환. curdir = current directory

# --------------------------------------------------------
# 13, 14 줄
_ = '\\' if platform.system() == 'Windows' else '/'
print(_)    # \
# 시스템 플랫폼이 윈도우일 때는 파일경로에 \\를 사용 , 리눅스 일 때는 / 사용

if root_dir[len(root_dir) - 1] != _: root_dir += _
print(root_dir) # d:\yespeechcode\kochat\yr_hack\
# 위에서 받은 root_dir이 \로 아닌 문자로 끝나면 끝에 \를 붙여주기

d:\yespeechcode\kochat\yr_hack
\
d:\yespeechcode\kochat\yr_hack\


## 16 줄, BASE  
하나씩 주석 달기

In [ ]:
BASE = {
    'root_dir': root_dir.format(_=_),  # 백엔드 루트경로
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'vector_size': 128,  # 단어 벡터 사이즈
    'batch_size': 512,  # 미니배치 사이즈
    'max_len': 8,  # 문장의 최대 길이 (패드 시퀀싱)
    'delimeter': _,  # OS에 따른 폴더 delimeter

    'PAD': 0,  # PAD 토큰 값 (전체가 0인 벡터)
    'OOV': 1  # OOV 토큰 값 (전체가 1인 벡터)
}

In [45]:
# ===================
root_dir.format(_=_)
# ===================

# 현재 있는 경로를 백엔드로 지정
# kochat_config.py 에서 28번 DATA에 사용

'd:\\yespeechcode\\kochat\\yr_hack\\'

In [ ]:
# ===================
'device': 'cuda' if torch.cuda.is_available() else 'cpu'
# ===================

# cuda gpu 사용가능하면 cuda 아니면 cpu

In [ ]:
# ===================
'vector_size': 128,  # 단어 벡터 사이즈
# ===================

# word_embedding의 대표적인 기법인 Word2vec 과 fasttext
# 참고: https://inspiringpeople.github.io/data%20analysis/word_embedding/
# --------------------------------------------------------
# kochat/model/embed/word2vec.py 19번
class Word2Vec(Word2Vec):
    def __init__(self):
        """
        Gensim Word2Vec 모델의 Wrapper 클래스입니다.
        """
        super().__init__(size=self.vector_size,         # word 벡터 차원(embedding size)
                         window=self.window_size,       # 현재 단어와 예측 단어의 최대 거리
                         workers=self.workers,          # 모델 생성시 사용할 thread 수
                         min_count=self.min_count,      # min_count 빈도수도다 낮은 빈도수 단어는 무시
                         iter=self.iter)                # 학습 횟수

# --------------------------------------------------------
# kochat/model/embed/fasttext.py 19번
class FastText(FastText):
    def __init__(self):
        """
        Gensim Word2Vec 모델의 Wrapper 클래스입니다.
        """
        super().__init__(size=self.vector_size,
                         window=self.window_size,
                         workers=self.workers,
                         min_count=self.min_count,
                         iter=self.iter)

# --------------------------------------------------------
# kochat/model/entity/lstm.py 21번
self.lstm = nn.LSTM(input_size=self.vector_size ... 
# kochat/model/intent/lstm.py 28번
self.lstm = nn.LSTM(input_size=self.vector_size ...

# --------------------------------------------------------
# kochat/proc/intent_classifier.py 64번
sample = torch.randn(1, self.max_len, self.vector_size)
# torch.randn() : 평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용해 생성
# 사이즈를 (1, max_len, vector_size)로 생성

# kochat/proc/entity_recognizer.py 144번
sample = torch.randn(1, self.max_len, self.vector_size)

# --------------------------------------------------------
# kochat/proc/gensim_embedder.py 94번
word_vector = torch.ones(self.vector_size) * self.OOV
# word_vactor 사이즈를 vector_size로 맞추고 OOV 값 곱하기          # 'OOV': 1

# --------------------------------------------------------
# kochat/data/preprocessor.py 49번
pad = torch.ones(self.max_len, self.vector_size) * self.PAD
# 문장이 max_len보다 짧으면 길이가 max_len인 0벡터를 만들기         # 'PAD': 0


In [ ]:
# ===================
'batch_size': 512,  # 미니배치 사이즈
# ===================

# --------------------------------------------------------
# kochat/model/entity/lstm.py 27번
param1 = torch.randn(self.layers * self.direction, batch_size, self.d_model).to(self.device)
# kochat/model/intent/lstm.py 34번
param1 = torch.randn(self.layers * self.direction, batch_size, self.d_model).to(self.device)
# entity, Intent Classification을 위한 LSTM 클래스

# --------------------------------------------------------
# kochat/data/dataset.py 290번
return DataLoader(
        dataset=TensorDataset(*tensors),
        batch_size=self.batch_size,
        shuffle=True,
        pin_memory=True     # True러 선언하면, 데이터로더는 Tensor를 CUDA 고정 메모리에 올립니다.
    )
# pytorch에서 제공하는 DaraLoader

In [ ]:
# ===================
'max_len': 8,  # 문장의 최대 길이 (패드 시퀀싱)
# ===================

# --------------------------------------------------------
# kochat/data/preprocessor.py 43번
def pad_sequencing(self, sequence: Tensor) -> tuple:
    """
    패드 시퀀싱 함수입니다.
    max_len보다 길이가 길면 자르고, 짧으면 뒤에 패딩(영벡터)를 추가합니다.
    엔티티 학습시에 CRF나 Masking 등을 이용하기 위해 각 문장의 길이가 필요합니다.
    패드 시퀀싱 단계에서는 어차피 길이를 세기 때문에 길이를 함께 반환합니다.
    :param sequence: 패드 시퀀싱할 문장입니다. (tensor로 이미 임베딩 된 문장)
    :return: 패드시퀀싱된 문장과 시퀀싱 전의 문장의 원래 길이
    """
    length = sequence.size()[0]
    if length > self.max_len:
        sequence = sequence[:self.max_len]
        length = self.max_len  # 마스킹시에 길이가 max_len 넘어가면 안됨
        # 문장이 max_len보다 길면 뒷부분을 자릅니다.
    else:
        pad = torch.ones(self.max_len, self.vector_size) * self.PAD
        for i in range(length):
            pad[i] = sequence[i]
        sequence = pad
        # 문장이 max_len보다 짧으면 길이가 max_len인 0벡터를 만들고
        # 데이터가 있던 인덱스에는 원래 데이터를 복사합니다
    return sequence, length

# --------------------------------------------------------
# kochat/loss/utils/masking.py 11번
# kochat/loss/masking.py 13번
class Masking(nn.Module):
    def __init__(self):
        """
        시퀀스 길이를 받아서 max_len 길이의 마스킹 벡터들의 집합을 만듭니다.
        e.g. sequence_length = 3,
        [True, True, True, False, False , ..., False]
        """
        super().__init__()
    def forward(self, sequence_length: Tensor) -> Tensor:
        batch_size = sequence_length.size(0)
        masks = []
        for i in range(batch_size):
            mask = torch.zeros(self.max_len, dtype=torch.uint8).to(self.device)
            # 전부다 0으로 된 마스킹 벡터 생성
            for j in range(sequence_length[i]):
                # seq length까지만 1로 만들어줌
                mask[j] = 1
            masks.append(mask.unsqueeze(0))
            # 마스크 배열에 넣어줌
        return torch.cat(masks, dim=0)
        # batchwise concatenation

# --------------------------------------------------------
# kochat/proc/intent_classifier.py 64번
# kochat/proc/entity_recognizer.py 144번
sample = torch.randn(1, self.max_len, self.vector_size)

In [ ]:
# ===================
'delimeter': _  # OS에 따른 폴더 delimeter(구분자)
# 윗 줄에서 window라면 \\으로 linux면 /으로 지정됨
# ===================

# --------------------------------------------------------
# kochat/proc/base_processor.py 27번
self.model_dir = self.model_dir + \
                 self.__class__.__name__ + \
                 self.delimeter

# --------------------------------------------------------
# kochat/utils/visualizer.py 281번
# kochat/proc/utils/visualizer.py 281번
f = open(self.model_dir + 'temp{_}{mode}.txt'.format(_=self.delimeter, mode=mode), 'r')
file = f.read()

## 29번 DATA

In [ ]:
DATA = {
    'data_ratio': 0.8,  # 학습\\검증 데이터 비율
    'raw_data_dir': BASE['root_dir'] + "data{_}raw{_}".format(_=_),  # 원본 데이터 파일 경로
    'ood_data_dir': BASE['root_dir'] + "data{_}ood{_}".format(_=_),  # out of distribution 데이터셋
    'intent_data_dir': BASE['root_dir'] + "data{_}intent_data.csv".format(_=_),  # 생성된 인텐트 데이터 파일 경로
    'entity_data_dir': BASE['root_dir'] + "data{_}entity_data.csv".format(_=_),  # 생성된 엔티티 데이터 파일 경로

    'NER_categories': ['DATE', 'LOCATION', 'RESTAURANT', 'PLACE'],  # 사용자 정의 태그
    'NER_tagging': ['B', 'E', 'I', 'S'],  # NER의 BEGIN, END, INSIDE, SINGLE 태그
    'NER_outside': 'O',  # NER의 O태그 (Outside를 의미)

In [ ]:
# ===================
'data_ratio': 0.8,  # 학습\\검증 데이터 비율
# ===================

# --------------------------------------------------------
# kochat/data/dataset.py 239번
def __split_data(self, dataset: list) -> tuple:
    """
    데이터셋을 학습용 / 검증용으로 나눕니다.
    Configuration에 적힌 split ratio를 기준으로 데이터를 쪼갭니다.
    :param dataset: 토큰화가 완료된 리스트 데이터셋
    :return: 분리가 완료된 (학습용 데이터, 검증용 데이터)
    """
    random.shuffle(dataset)  # 데이터 섞어주기
    split_point = int(len(dataset) * self.data_ratio)   #####
    train_dataset = dataset[:split_point]
    test_dataset = dataset[split_point:]
    return train_dataset, test_dataset

In [ ]:
# ===================
'raw_data_dir': BASE['root_dir'] + "data{_}raw{_}".format(_=_),  # 원본 데이터 파일 경로
'ood_data_dir': BASE['root_dir'] + "data{_}ood{_}".format(_=_),  # out of distribution 데이터셋
'intent_data_dir': BASE['root_dir'] + "data{_}intent_data.csv".format(_=_),  # 생성된 인텐트 데이터 파일 경로
'entity_data_dir': BASE['root_dir'] + "data{_}entity_data.csv".format(_=_),  # 생성된 엔티티 데이터 파일 경로
# ===================

# --------------------------------------------------------
# 파일 경로

In [ ]:
# ===================
'NER_categories': ['DATE', 'LOCATION', 'RESTAURANT', 'PLACE'],  # 사용자 정의 태그
# ===================

# --------------------------------------------------------
# kochat/app/scenario.py 42번
pre_defined_entity = [entity.lower() for entity in self.NER_categories]

# --------------------------------------------------------
# kochat/data/organizer.py 153번
label = [tag + '-' + cate
         for cate in self.NER_categories
         for tag in self.NER_tagging] + [self.NER_outside]
         # Config에서 지정한 라벨들의 조합 + Outside 라벨만 가질 수 있음

# entity 예 )
# 게다가 광명 우산 가져가야하니,O S-LOCATION O O
# 아 울산 이번 주 비 오니,O S-LOCATION B-DATE E-DATE O O

In [ ]:
# ===================
'NER_tagging': ['B', 'E', 'I', 'S'],  # NER의 BEGIN, END, INSIDE, SINGLE 태그
# ===================

# --------------------------------------------------------
# kochat/data/organizer.py 154번
label = [tag + '-' + cate
         for cate in self.NER_categories
         for tag in self.NER_tagging] + [self.NER_outside]
         # Config에서 지정한 라벨들의 조합 + Outside 라벨만 가질 수 있음

# entity 예 )
# 게다가 광명 우산 가져가야하니,O S-LOCATION O O
# 아 울산 이번 주 비 오니,O S-LOCATION B-DATE E-DATE O O

In [ ]:
# ===================
'NER_outside': 'O',  # NER의 O태그 (Outside를 의미)
# ===================

# --------------------------------------------------------
# kochat/data/organizer.py 154번
label = [tag + '-' + cate
         for cate in self.NER_categories
         for tag in self.NER_tagging] + [self.NER_outside]
         # Config에서 지정한 라벨들의 조합 + Outside 라벨만 가질 수 있음

# entity 예 )
# 게다가 광명 우산 가져가야하니,O S-LOCATION O O
# 아 울산 이번 주 비 오니,O S-LOCATION B-DATE E-DATE O O

# --------------------------------------------------------
# kochat/data/preprocessor.py 79번
def label_sequencing(self, entity_label: Tensor, entity_dict: dict) -> Tensor:
    """
    엔티티 라벨의 경우에는 라벨도 각각 길이가 다르게 됩니다.
    e.g. [O, DATE, O](size=3),  [DATE, O, O, O](size=4)
    길이가 다른 벡터들을 텐서의 형태로 만들려면 이들의 길이도 같아야합니다.
    :param entity_label: 한 문장의 엔티티 라벨 (1차원)
    :param entity_dict: 딕셔너리를 이용해 빈부분에 outside 태그를 넣습니다.
    :return: 패드시퀀싱 된 엔티티 라벨
    """
    length = entity_label.size()[0]

    if length > self.max_len:
        entity_label = entity_label[:self.max_len]
        # 길이가 max_len보다 길면 뒷부분을 자릅니다.

    else:
        pad = torch.ones(self.max_len, dtype=torch.int64)
        outside_tag = entity_dict[self.NER_outside]
        pad = pad * outside_tag  # 'O' 태그가 맵핑된 숫자       ##########
        # [1, 1, ..., 1] * 'O' => ['O', 'O', ... , 'O']

        for i in range(length):
            pad[i] = entity_label[i]
        entity_label = pad
        # 문장이 max_len보다 짧으면 길이가 max_len인 'O'벡터를 만들고
        # 데이터가 있던 인덱스에는 원래 데이터를 복사합니다

    return entity_label.unsqueeze(0)